In [33]:
import ROOT as rt
from matplotlib import pyplot as plt
import numpy as np
from ast import literal_eval as make_tuple

In [34]:
fname = input("Path to the ROOT file:")

Path to the ROOT file:/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_xlayer1.root


In [35]:
print("Using ROOT file:",fname)
inFile = rt.TFile(fname, "READ")

Using ROOT file: /user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_xlayer1.root


In [36]:
inFile.ls()

TFile**		/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_xlayer1.root	
 TFile*		/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_xlayer1.root	
  KEY: TTree	tree;1	tree


In [37]:
treeName = input("Name of the TTree:")

Name of the TTree:tree


In [38]:
myTree = inFile.Get(treeName)

In [39]:
N_det_unit = int(input("Number of deterctor units:"))

Number of deterctor units:2


In [40]:
N_fov=make_tuple(input("FOV Dimensions:"))

FOV Dimensions:200,200,100


In [41]:
N_fov = tuple(int(value) for value in N_fov)

In [42]:
hmapAll = rt.TH3F("hmapAll","hampAll",N_det_unit,0,N_det_unit,N_fov[0],-N_fov[0]//2,N_fov[0]//2,N_fov[1],-N_fov[1]//2,N_fov[1]//2)
myTree.Draw("(layerID + crystalID*8 + submoduleID*32 + rsectorID*128):sourcePosY:sourcePosX>>hmapAll","","goff")

7142

In [43]:
ntotal=int(input("Total number of gamma rays:"))

Total number of gamma rays:100


In [44]:
sysmat = np.reshape(np.array(hmapAll),(N_det_unit+2,N_fov[0]+2,N_fov[1]+2))[1:-1,1:-1,1:-1]/ntotal

In [46]:
#sysmat=sysmat.reshape((N_det_unit,np.prod(N_fov)))
sysmat=sysmat.reshape((N_det_unit,-1))

In [47]:
print("N_det_unit:", N_det_unit)
print("N_fov:", N_fov)
print("Size of sysmat:", sysmat.size)


N_det_unit: 2
N_fov: (200, 200, 100)
Size of sysmat: 80000


In [50]:
det_id = int(input(f"Detector unit index [0-{N_det_unit}]:"))

Detector unit index [0-2]:0


In [54]:
fig, ax = plt.subplots(figsize=(10, 9))
imshow_obj = ax.imshow(sysmat[det_id].reshape(N_fov),cmap='turbo',origin='lower')
cbar=plt.colorbar(imshow_obj)
plt.suptitle(f"PPDF of detector unit {det_id}")
ax.set_xlabel("FOV Voxel X (mm)")
ax.set_ylabel("FOV Voxel Y (mm) ")
fig.tight_layout()

ValueError: cannot reshape array of size 40000 into shape (200,200,100)

In [60]:
# Check if sysmat[det_id] has the right number of elements for the reshape
if sysmat[det_id].size == (N_fov[0] * N_fov[1]):
    reshaped_data = sysmat[det_id].reshape(N_fov[0], N_fov[1])
    print("Y")
else:
    print(f"Warning: Expected shape doesn't match. Size: {sysmat[det_id].size}, Expected: {N_fov[0] * N_fov[1]}")

# Now proceed to plot if reshaping was successful
fig, ax = plt.subplots(figsize=(10, 9))
imshow_obj = ax.imshow(reshaped_data, cmap='turbo', origin='lower', vmin=0, vmax=np.max(reshaped_data))

cbar = plt.colorbar(imshow_obj)
plt.suptitle(f"PPDF of detector unit {det_id}")
ax.set_xlabel("FOV Voxel X (mm)")
ax.set_ylabel("FOV Voxel Y (mm) ")
fig.tight_layout()
plt.show()


Y


In [61]:
print("Min value in reshaped_data:", np.min(reshaped_data))
print("Max value in reshaped_data:", np.max(reshaped_data))


Min value in reshaped_data: 0.0
Max value in reshaped_data: 0.0


In [1]:
import ROOT as rt
import numpy as np

# Load the ROOT file
root_filename = "/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged.root"  # Change this to your actual file path
root_file = rt.TFile(root_filename)

# Load the tree from the ROOT file
tree = root_file.Get("myTree")  # Replace 'myTree' with the actual tree name in your ROOT file

# Find out the number of entries (events) in the tree
n_entries = tree.GetEntries()
print(f"Number of entries in the tree: {n_entries}")

# Extract the branches from the tree
detector_branch = tree.GetBranch("detectorID")  # Example branch name for detectors
sourcePosX_branch = tree.GetBranch("sourcePosX")  # Example branch name for source positions
sourcePosY_branch = tree.GetBranch("sourcePosY")
sourcePosZ_branch = tree.GetBranch("sourcePosZ")

# Create empty lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Loop through all the entries in the tree and extract relevant data
for entry in range(n_entries):
    tree.GetEntry(entry)
    
    detector_ids.append(tree.detectorID)
    source_pos_x.append(tree.sourcePosX)
    source_pos_y.append(tree.sourcePosY)
    source_pos_z.append(tree.sourcePosZ)

# Convert lists to numpy arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Infer N_det_unit and N_fov automatically from the data
N_det_unit = len(np.unique(detector_ids))
N_fov_x = int(source_pos_x.max() - source_pos_x.min())
N_fov_y = int(source_pos_y.max() - source_pos_y.min())
N_fov_z = int(source_pos_z.max() - source_pos_z.min())
N_fov = (N_fov_x, N_fov_y, N_fov_z)

print(f"Inferred N_det_unit: {N_det_unit}")
print(f"Inferred FOV (x, y, z): {N_fov}")

# Create an empty system matrix
sysmat = np.zeros((N_det_unit, N_fov_x * N_fov_y * N_fov_z))

# Fill the system matrix with data
for i in range(n_entries):
    det_id = detector_ids[i]
    x_bin = int(source_pos_x[i] - source_pos_x.min())
    y_bin = int(source_pos_y[i] - source_pos_y.min())
    z_bin = int(source_pos_z[i] - source_pos_z.min())
    
    # Flatten the 3D index to 1D for the system matrix
    index = x_bin + y_bin * N_fov_x + z_bin * N_fov_x * N_fov_y
    sysmat[det_id, index] += 1

# Normalize the system matrix by total number of events (gamma rays)
sysmat = sysmat / n_entries

# Optionally save the system matrix to a file
np.save("system_matrix.npy", sysmat)

# Close the ROOT file
root_file.Close()

print("System matrix shape:", sysmat.shape)
print("System matrix extraction and conversion completed!")


Welcome to JupyROOT 6.26/10


AttributeError: 'TObject' object has no attribute 'GetEntries'

In [1]:
import ROOT as rt
import numpy as np

# Load the ROOT file
root_filename = "/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged.root"  # Change this to your actual file path
root_file = rt.TFile(root_filename)

# Check if the file was successfully opened
if root_file.IsZombie():
    print("Error: Could not open the ROOT file.")
    exit()

# List the available keys (trees, histograms, etc.) in the file to help identify the correct tree name
print("Keys in the ROOT file:")
root_file.ls()

# Load the tree from the ROOT file
tree_name = "tree"  # Replace with the actual tree name after checking the output from root_file.ls()
tree = root_file.Get(tree_name)

# Check if the tree was successfully loaded
if not tree:
    print(f"Error: Tree '{tree_name}' not found in the file.")
    root_file.Close()
    exit()

# Now you can proceed to extract entries from the tree
n_entries = tree.GetEntries()
print(f"Number of entries in the tree: {n_entries}")

# Extract the branches from the tree (you can check the branch names using tree.Print())
detector_branch = tree.GetBranch("detectorID")  # Example branch name, adjust if needed
sourcePosX_branch = tree.GetBranch("sourcePosX")  # Example branch name
sourcePosY_branch = tree.GetBranch("sourcePosY")
sourcePosZ_branch = tree.GetBranch("sourcePosZ")

# Create empty lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Loop through all the entries in the tree and extract relevant data
for entry in range(n_entries):
    tree.GetEntry(entry)
    
    detector_ids.append(tree.detectorID)
    source_pos_x.append(tree.sourcePosX)
    source_pos_y.append(tree.sourcePosY)
    source_pos_z.append(tree.sourcePosZ)

# Convert lists to numpy arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Infer N_det_unit and N_fov automatically from the data
N_det_unit = len(np.unique(detector_ids))
N_fov_x = int(source_pos_x.max() - source_pos_x.min())
N_fov_y = int(source_pos_y.max() - source_pos_y.min())
N_fov_z = int(source_pos_z.max() - source_pos_z.min())
N_fov = (N_fov_x, N_fov_y, N_fov_z)

print(f"Inferred N_det_unit: {N_det_unit}")
print(f"Inferred FOV (x, y, z): {N_fov}")

# Create an empty system matrix
sysmat = np.zeros((N_det_unit, N_fov_x * N_fov_y * N_fov_z))

# Fill the system matrix with data
for i in range(n_entries):
    det_id = detector_ids[i]
    x_bin = int(source_pos_x[i] - source_pos_x.min())
    y_bin = int(source_pos_y[i] - source_pos_y.min())
    z_bin = int(source_pos_z[i] - source_pos_z.min())
    
    # Flatten the 3D index to 1D for the system matrix
    index = x_bin + y_bin * N_fov_x + z_bin * N_fov_x * N_fov_y
    sysmat[det_id, index] += 1

# Normalize the system matrix by total number of events (gamma rays)
sysmat = sysmat / n_entries

# Optionally save the system matrix to a file
np.save("system_matrix.npy", sysmat)

# Close the ROOT file
root_file.Close()

print("System matrix shape:", sysmat.shape)
print("System matrix extraction and conversion completed!")


Welcome to JupyROOT 6.26/10
Keys in the ROOT file:
TFile**		/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged.root	
 TFile*		/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged.root	
  KEY: TTree	tree;1	tree
Number of entries in the tree: 24495


AttributeError: 'TTree' object has no attribute 'detectorID'

In [1]:
import ROOT as rt

# Load the ROOT file
root_filename = "/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged.root"
root_file = rt.TFile(root_filename)

# Check if the file was successfully opened
if root_file.IsZombie():
    print("Error: Could not open the ROOT file.")
    exit()

# Load the tree from the ROOT file
tree_name = "tree"
tree = root_file.Get(tree_name)

# Check if the tree was successfully loaded
if not tree:
    print(f"Error: Tree '{tree_name}' not found in the file.")
    root_file.Close()
    exit()

# Print the available branches in the tree
print("Available branches in the tree:")
tree.Print()

# Close the ROOT file
root_file.Close()


Welcome to JupyROOT 6.26/10
Available branches in the tree:
******************************************************************************
*Tree    :tree      : tree                                                   *
*Entries :    24495 : Total =         2816800 bytes  File  Size =     981127 *
*        :          : Tree compression factor =   2.87                       *
******************************************************************************
*Br    0 :runID     : runID/I                                                *
*Entries :    24495 : Total  Size=      98825 bytes  File Size  =       1176 *
*Baskets :        5 : Basket Size=      32000 bytes  Compression=  83.62     *
*............................................................................*
*Br    1 :eventID   : eventID/I                                              *
*Entries :    24495 : Total  Size=      98843 bytes  File Size  =      44419 *
*Baskets :        5 : Basket Size=      32000 bytes  Compression=   2.2

In [ ]:
import ROOT as rt
import numpy as np

# Load the ROOT file
root_filename = "/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged.root"
root_file = rt.TFile(root_filename)

# Load the tree from the ROOT file
tree_name = "tree"
tree = root_file.Get(tree_name)

# Check if the tree was successfully loaded
if not tree:
    print(f"Error: Tree '{tree_name}' not found in the file.")
    root_file.Close()
    exit()

# Prepare arrays to hold the system matrix data
source_pos_x = []
source_pos_y = []
source_pos_z = []
detector_ids = []

# Iterate over the entries in the tree and collect data
n_entries = tree.GetEntries()
for entry in range(n_entries):
    tree.GetEntry(entry)

    source_pos_x.append(tree.sourcePosX)
    source_pos_y.append(tree.sourcePosY)
    source_pos_z.append(tree.sourcePosZ)
    
    # Combine detector info: use IDs like rsectorID, moduleID, submoduleID, crystalID, and layerID
    detector_ids.append((tree.rsectorID, tree.moduleID, tree.submoduleID, tree.crystalID, tree.layerID))

# Convert lists to NumPy arrays
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)
detector_ids = np.array(detector_ids)

# You can now use source positions and detector IDs to build the system matrix
# Example: system matrix as a 2D array (detectors by sources)
# For simplicity, we'll concatenate the source positions into a single array
sources = np.vstack((source_pos_x, source_pos_y, source_pos_z)).T

# Create the system matrix (number of detectors x number of sources)
system_matrix = np.zeros((detector_ids.shape[0], sources.shape[0]))

# Fill the system matrix with dummy values (you can use a real metric here)
for i in range(detector_ids.shape[0]):
    # Use some metric to fill in the system matrix
    system_matrix[i] = np.linalg.norm(sources - sources[i], axis=1)  # Example using Euclidean distance

print("System matrix generated.")
root_file.Close()


In [1]:
import ROOT as rt
import numpy as np

# Load the ROOT file
root_filename = "/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged.root"
root_file = rt.TFile(root_filename)

# Load the tree from the ROOT file
tree_name = "tree"
tree = root_file.Get(tree_name)

# Check if the tree was successfully loaded
if not tree:
    print(f"Error: Tree '{tree_name}' not found in the file.")
    root_file.Close()
    exit()

n_entries = tree.GetEntries()
batch_size = 1000  # Process data in smaller batches

# Pre-allocate or use lists to accumulate data
source_pos_x, source_pos_y, source_pos_z = [], [], []
detector_ids = []

for start in range(0, n_entries, batch_size):
    end = min(start + batch_size, n_entries)
    for entry in range(start, end):
        tree.GetEntry(entry)

        # Store the data incrementally
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
        detector_ids.append((tree.rsectorID, tree.moduleID, tree.submoduleID, tree.crystalID, tree.layerID))

    # Optionally, process or save batches to avoid memory overload
    print(f"Processed entries {start} to {end}")

# Once done, convert to NumPy arrays
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)
detector_ids = np.array(detector_ids)

print("Data processed and converted to arrays.")
root_file.Close()


Welcome to JupyROOT 6.26/10
Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.


In [2]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/SPEBT/mcsim/macro/SPEBT.Singles_merged_60s.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.", system_matrix)

# Plotting a slice of the system matrix for a selected detector
det_id = int(input(f"Select detector ID [0-{len(np.unique(detector_ids)) - 1}]: "))
voxel_slice = system_matrix[det_id].reshape(fov_shape)

# Plot an example 2D slice (z=middle of the FOV)
z_slice = voxel_slice[:, :, fov_shape[2] // 2]

fig, ax = plt.subplots(figsize=(10, 9))
imshow_obj = ax.imshow(z_slice, cmap='turbo', origin='lower')
cbar = plt.colorbar(imshow_obj)
plt.suptitle(f"PPDF of detector unit {det_id}")
ax.set_xlabel("FOV Voxel X (mm)")
ax.set_ylabel("FOV Voxel Y (mm)")
fig.tight_layout()
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated.
Select detector ID [0-3]: 0


In [16]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.",system_matrix)

for i in system_matrix:
    print(np.unique(i))
    
for det_id in np.unique(detector_ids):
    # Plotting a slice of the system matrix for a selected detector
    #det_id = int(input(f"Select detector ID [0-{len(np.unique(detector_ids)) - 1}]: "))
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Plot an example 2D slice (z=middle of the FOV)
    z_slice = voxel_slice[:, :, fov_shape[2] // 2]

    fig, ax = plt.subplots(figsize=(10, 9))
    imshow_obj = ax.imshow(z_slice, cmap='turbo', origin='lower')
    cbar = plt.colorbar(imshow_obj)
    plt.suptitle(f"PPDF of detector unit {det_id}")
    ax.set_xlabel("FOV Voxel X (mm)")
    ax.set_ylabel("FOV Voxel Y (mm)")
    fig.tight_layout()

    # Save the plot as a PNG file
    output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_detector_{}.png".format(det_id)
    plt.savefig(output_path)

    print(f"Plot saved to {output_path}")
    plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated. [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 1. 2.]
[0. 1. 2.]
[0. 1. 2.

In [14]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged_60s.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid

# Create binning for X, Y, Z based on source position ranges
x_bins = np.linspace(-90, 90, fov_shape[0])
y_bins = np.linspace(-90, 90, fov_shape[1])
z_bins = np.linspace(-0.5, 0.5, fov_shape[2])

# Initialize the system matrix
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1

    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Print the system matrix
print("System Matrix:")
print(system_matrix)
for i in system_matrix:
    print(np.unique(i))

# Plotting for each detector
for det_id in np.unique(detector_ids):
    # Reshape the system matrix for the selected detector
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Plot an example 2D slice (z = middle of the FOV)
    z_slice = voxel_slice[:, :, fov_shape[2] // 2]

    fig, ax = plt.subplots(figsize=(10, 9))
    imshow_obj = ax.imshow(z_slice, cmap='turbo', origin='lower')
    cbar = plt.colorbar(imshow_obj)
    plt.suptitle(f"PPDF of detector unit {det_id}")
    ax.set_xlabel("FOV Voxel X (mm)")
    ax.set_ylabel("FOV Voxel Y (mm)")
    fig.tight_layout()

    # Save the plot as a PNG file
    output_path = f"/user/tridevme/MCSIM/MCSIM/macro/system_matrix_detector_60s_{det_id}.png"
    plt.savefig(output_path)

    print(f"Plot saved to {output_path}")
    plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated.
System Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 1. 2.]
[0. 1

In [2]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged_60s.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# FOV parameters with coarser bins (adjust based on your needs)
fov_shape = (50, 50, 25)  # Coarser shape (x, y, z) for the FOV grid

# Create binning for X, Y, Z based on source position ranges
x_bins = np.linspace(-90, 90, fov_shape[0])
y_bins = np.linspace(-90, 90, fov_shape[1])
z_bins = np.linspace(-0.5, 0.5, fov_shape[2])

# Initialize the system matrix
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1

    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated with re-binning.")

# Print the system matrix for verification
print("System Matrix after re-binning:")
print(system_matrix)
for i in system_matrix:
    print(np.unique(i))

# Plotting for each detector
for det_id in np.unique(detector_ids):
    # Reshape the system matrix for the selected detector
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Plot an example 2D slice (z = middle of the FOV)
    z_slice = voxel_slice[:, :, fov_shape[2] // 2]

    fig, ax = plt.subplots(figsize=(10, 9))
    imshow_obj = ax.imshow(z_slice, cmap='turbo', origin='lower')
    cbar = plt.colorbar(imshow_obj)
    plt.suptitle(f"PPDF of detector unit {det_id} (re-binned)")
    ax.set_xlabel("FOV Voxel X (mm)")
    ax.set_ylabel("FOV Voxel Y (mm)")
    fig.tight_layout()

    # Save the plot as a PNG file
    output_path = f"/user/tridevme/MCSIM/MCSIM/macro/system_matrix_detector_rebinned_{det_id}.png"
    plt.savefig(output_path)

    print(f"Re-binned plot saved to {output_path}")
    plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated with re-binning.
System Matrix after re-binning:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0

In [3]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Fix j (i.e., choose a specific voxel)
fixed_voxel_idx = 7000  # Example voxel index, adjust based on your application
detector_contributions = system_matrix[:, fixed_voxel_idx]

# Plot the contributions of detectors for the fixed voxel
fig, ax = plt.subplots(figsize=(10, 7))
ax.bar(np.arange(len(detector_contributions)), detector_contributions, color='blue')
ax.set_xlabel("Detector ID")
ax.set_ylabel("Contribution to fixed voxel")
plt.title(f"System matrix contributions for fixed voxel j={fixed_voxel_idx}")
plt.grid(True)

# Save the plot
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_fixed_voxel_{}.png".format(fixed_voxel_idx)
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated.
Plot saved to /user/tridevme/MCSIM/MCSIM/macro/system_matrix_fixed_voxel_7000.png


In [6]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    
    # Plot a 2D slice at the middle of the z-axis
    z_slice = voxel_slice[:, :, fov_shape[2] // 2]

    # Use a logarithmic color scale for the plot
    im = ax.imshow(z_slice, cmap='viridis', origin='lower', norm=LogNorm())
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view x-axis (mm)')
    ax.set_ylabel('Field of view y-axis (mm)')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated.
Plot saved to /user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors.png


In [2]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    
    # Plot a 2D slice at the middle of the z-axis
    z_slice = voxel_slice[:, :, fov_shape[2] // 2]

    # Use a logarithmic color scale for the plot and set lower limit for visibility
    im = ax.imshow(z_slice, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-2, vmax=np.max(z_slice)))
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view x-axis (mm)')
    ax.set_ylabel('Field of view y-axis (mm)')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Plot saved to /user/

In [18]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Plot a 2D slice averaged over a few z-slices instead of just the middle one
    z_slice = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)

    # Use a logarithmic color scale for the plot and set lower limit for visibility
    im = ax.imshow(z_slice, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(z_slice) or 1))
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view x-axis (mm)')
    ax.set_ylabel('Field of view y-axis (mm)')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_works.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Plot saved to /user/

In [9]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    
    # Plot a 2D slice at the middle of the z-axis
    z_slice = voxel_slice[:, :, fov_shape[2] // 2]

    # Use a logarithmic color scale for the plot and set lower limit for visibility
    im = ax.imshow(z_slice, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-2, vmax=np.max(z_slice)))
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view x-axis (mm)')
    ax.set_ylabel('Field of view y-axis (mm)')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Plot saved to /user/

In [10]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    
    # Try using different slices along the z-axis (e.g., quarter, mid, and three-quarters)
    z_slice_mid = voxel_slice[:, :, fov_shape[2] // 2]  # Middle slice
    z_slice_front = voxel_slice[:, :, fov_shape[2] // 4]  # Front slice
    z_slice_back = voxel_slice[:, :, 3 * fov_shape[2] // 4]  # Back slice
    
    # Stack slices for better data visualization (average of different depths)
    z_slice_avg = (z_slice_mid + z_slice_front + z_slice_back) / 3

    # Use a logarithmic color scale for the plot and set lower limit for visibility
    im = ax.imshow(z_slice_avg, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(z_slice_avg) or 1))
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view x-axis (mm)')
    ax.set_ylabel('Field of view y-axis (mm)')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Plot saved to /user/

In [1]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged_cone_6s.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    
    # Try using different slices along the Y-axis (e.g., quarter, mid, and three-quarters)
    y_slice_mid = voxel_slice[:, fov_shape[1] // 2, :]  # Middle slice along Y-axis
    y_slice_front = voxel_slice[:, fov_shape[1] // 4, :]  # Front slice along Y-axis
    y_slice_back = voxel_slice[:, 3 * fov_shape[1] // 4, :]  # Back slice along Y-axis
    
    # Stack slices for better data visualization (average of different depths)
    y_slice_avg = (y_slice_mid + y_slice_front + y_slice_back) / 3

    # Use a logarithmic color scale for the plot and set lower limit for visibility
    im = ax.imshow(y_slice_avg, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(y_slice_avg) or 1))
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view x-axis (mm)')
    ax.set_ylabel('Field of view z-axis (mm)')  # Z-axis now becomes the Y-label for visualization

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_cone_Y_slice.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Welcome to JupyROOT 6.26/10
Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix gen

In [2]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged_cone_6s.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    
    # Try using different slices along the X-axis (e.g., quarter, mid, and three-quarters)
    x_slice_mid = voxel_slice[fov_shape[0] // 2, :, :]  # Middle slice along X-axis
    x_slice_front = voxel_slice[fov_shape[0] // 4, :, :]  # Front slice along X-axis
    x_slice_back = voxel_slice[3 * fov_shape[0] // 4, :, :]  # Back slice along X-axis
    
    # Stack slices for better data visualization (average of different depths)
    x_slice_avg = (x_slice_mid + x_slice_front + x_slice_back) / 3

    # Use a logarithmic color scale for the plot and set lower limit for visibility
    im = ax.imshow(x_slice_avg, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(x_slice_avg) or 1))
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view y-axis (mm)')  # Y-axis now becomes the X-label for visualization
    ax.set_ylabel('Field of view z-axis (mm)')  # Z-axis remains the Y-label for visualization

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = plt.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_cone_X_slice.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Plot saved to /user/

In [4]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged_cone_6s.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        # Append data to the lists
        detector_ids.append(tree.crystalID)  # Adjust according to your detector ID
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters (adjust based on your needs)
fov_shape = (200, 200, 100)  # Example shape (x, y, z) for the FOV grid
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    # Find the voxel (FOV index) corresponding to each event
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    # Ensure valid indices
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Select multiple detectors (or one specific detector) for visualization
selected_detectors = [0, 1, 2, 3]  # Adjust according to your setup
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))  # 2 rows, 4 columns

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]  # Arrange subplots in a grid
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    
    # Try using different slices along the Z-axis (e.g., quarter, mid, and three-quarters)
    z_slice_mid = voxel_slice[:, :, fov_shape[2] // 2]  # Middle slice along Z-axis
    z_slice_front = voxel_slice[:, :, fov_shape[2] // 4]  # Front slice along Z-axis
    z_slice_back = voxel_slice[:, :, 3 * fov_shape[2] // 4]  # Back slice along Z-axis
    
    # Stack slices for better data visualization (average of different depths)
    z_slice_avg = (z_slice_mid + z_slice_front + z_slice_back) / 3

    # Use a logarithmic color scale for the plot and set lower limit for visibility
    im = ax.imshow(z_slice_avg, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(z_slice_avg) or 1))
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel('Field of view x-axis (mm)')  # X-axis now becomes the X-label for visualization
    ax.set_ylabel('Field of view y-axis (mm)')  # Y-axis remains the Y-label for visualization

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # Colorbar axis
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot as a PNG file
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_cone_Z_slice.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Plot saved to /user/

In [12]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged_cone_6s.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters
fov_shape = (200, 200, 100)
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Ask user for axis input
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    if axis == 'Z':
        z_slice_mid = voxel_slice[:, :, fov_shape[2] // 2]
        z_slice_front = voxel_slice[:, :, fov_shape[2] // 4]
        z_slice_back = voxel_slice[:, :, 3 * fov_shape[2] // 4]
        slice_avg = (z_slice_mid + z_slice_front + z_slice_back) / 3
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_cone_Z_slice.png"

    elif axis == 'X':
        x_slice_mid = voxel_slice[fov_shape[0] // 2, :, :]
        x_slice_front = voxel_slice[fov_shape[0] // 4, :, :]
        x_slice_back = voxel_slice[3 * fov_shape[0] // 4, :, :]
        slice_avg = (x_slice_mid + x_slice_front + x_slice_back) / 3
        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_cone_X_slice.png"

    elif axis == 'Y':
        y_slice_mid = voxel_slice[:, fov_shape[1] // 2, :]
        y_slice_front = voxel_slice[:, fov_shape[1] // 4, :]
        y_slice_back = voxel_slice[:, 3 * fov_shape[1] // 4, :]
        slice_avg = (y_slice_mid + y_slice_front + y_slice_back) / 3
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_cone_Y_slice.png"

    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    im = ax.imshow(slice_avg, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(slice_avg) or 1))
    ax.set_title(f'Detector {det_id}')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()

Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Enter the axis for s

In [30]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# FOV parameters
fov_shape = (200, 200, 100)  # Example shape (x, y, z)
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Bin source positions into FOV and fill the system matrix
x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Choose slice based on user input
    if axis == 'Z':
        slice_data = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_works_Z_slice.png"
    
    elif axis == 'X':
        slice_data = np.mean(voxel_slice[fov_shape[0] // 4:3 * fov_shape[0] // 4, :, :], axis=0)
        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_works_X_slice.png"
    
    elif axis == 'Y':
        slice_data = np.mean(voxel_slice[:, fov_shape[1] // 4:3 * fov_shape[1] // 4, :], axis=1)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_works_Y_slice.png"
    
    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    # Use a logarithmic color scale for the plot
    im = ax.imshow(slice_data, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(slice_data) or 1))
    ax.set_title(f'Detector {det_id}')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Enter the axis for s

In [29]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# Define a larger FOV to zoom out (adjust the values to zoom out further if needed)
zoom_factor = 1  # Increase this to zoom out more
fov_shape = (200 * zoom_factor, 200 * zoom_factor, 100 * zoom_factor)  # Adjusted FOV size
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins for zoomed-out FOV
x_bins = np.linspace(np.min(source_pos_x) - 50, np.max(source_pos_x) + 50, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50, np.max(source_pos_y) + 50, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50, np.max(source_pos_z) + 50, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Choose slice based on user input
    if axis == 'Z':
        slice_data = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Z_slice_zoomed_out.png"
    
    elif axis == 'X':
        slice_data = np.mean(voxel_slice[fov_shape[0] // 4:3 * fov_shape[0] // 4, :, :], axis=0)
        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_zoomed_out.png"
    
    elif axis == 'Y':
        slice_data = np.mean(voxel_slice[:, fov_shape[1] // 4:3 * fov_shape[1] // 4, :], axis=1)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Y_slice_zoomed_out.png"
    
    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    # Use a logarithmic color scale for the plot
    im = ax.imshow(slice_data, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(slice_data) or 1))
    ax.set_title(f'Detector {det_id}')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Enter the axis for s

In [1]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# Define a FOV size that scales with the zoom factor
zoom_factor = 1  # Use values < 1 to zoom in, values > 1 to zoom out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Choose slice based on user input
    if axis == 'Z':
        slice_data = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Z_slice_zoomed.png"
    
    elif axis == 'X':
        slice_data = np.mean(voxel_slice[fov_shape[0] // 4:3 * fov_shape[0] // 4, :, :], axis=0)
        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_zoomed.png"
    
    elif axis == 'Y':
        slice_data = np.mean(voxel_slice[:, fov_shape[1] // 4:3 * fov_shape[1] // 4, :], axis=1)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Y_slice_zoomed.png"
    
    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    # Use a logarithmic color scale for the plot
    im = ax.imshow(slice_data, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(slice_data) or 1))
    ax.set_title(f'Detector {det_id}')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Welcome to JupyROOT 6.26/10
Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix gen

In [41]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# Define a FOV size that scales with the zoom factor
zoom_factor = 0.8  # Adjust this value to zoom in or out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Debugging: Inspect voxel data for each detector
selected_detectors = [0, 1, 2, 3]  # Modify as needed

for det_id in selected_detectors:
    print(f"\nInspecting Detector {det_id}")
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    print(f"Voxel slice shape: {voxel_slice.shape}")
    
    # Visualize raw voxel data in X-Y, Y-Z, and X-Z planes
    plt.figure(figsize=(18, 5))
    
    # X-Y plane (sum across Z)
    plt.subplot(1, 3, 1)
    plt.imshow(voxel_slice.sum(axis=2), cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - X-Y Plane (sum across Z)')
    plt.colorbar(label='Counts')
    
    # Y-Z plane (sum across X)
    plt.subplot(1, 3, 2)
    plt.imshow(voxel_slice.sum(axis=0), cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - Y-Z Plane (sum across X)')
    plt.colorbar(label='Counts')
    
    # X-Z plane (sum across Y)
    plt.subplot(1, 3, 3)
    plt.imshow(voxel_slice.sum(axis=1), cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - X-Z Plane (sum across Y)')
    plt.colorbar(label='Counts')
    
    plt.tight_layout()
    plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.

Inspecting Detector

In [42]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# Define a FOV size that scales with the zoom factor
zoom_factor = 0.8  # Adjust this value to zoom in or out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Debugging: Inspect voxel data for each detector
selected_detectors = [0, 1, 2, 3]  # Modify as needed

for det_id in selected_detectors:
    print(f"\nInspecting Detector {det_id}")
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    print(f"Voxel slice shape: {voxel_slice.shape}")
    
    # Calculate total counts for each plane
    xy_counts = voxel_slice.sum(axis=2)
    yz_counts = voxel_slice.sum(axis=0)
    xz_counts = voxel_slice.sum(axis=1)

    print(f"Total counts in X-Y Plane: {np.sum(xy_counts)}")
    print(f"Total counts in Y-Z Plane: {np.sum(yz_counts)}")
    print(f"Total counts in X-Z Plane: {np.sum(xz_counts)}")

    # Visualize raw voxel data in X-Y, Y-Z, and X-Z planes
    plt.figure(figsize=(18, 5))
    
    # X-Y plane (sum across Z)
    plt.subplot(1, 3, 1)
    plt.imshow(xy_counts, cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - X-Y Plane (sum across Z)')
    plt.colorbar(label='Counts')
    
    # Y-Z plane (sum across X)
    plt.subplot(1, 3, 2)
    plt.imshow(yz_counts, cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - Y-Z Plane (sum across X)')
    plt.colorbar(label='Counts')
    
    # X-Z plane (sum across Y)
    plt.subplot(1, 3, 3)
    plt.imshow(xz_counts, cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - X-Z Plane (sum across Y)')
    plt.colorbar(label='Counts')
    
    plt.tight_layout()
    plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.

Inspecting Detector

In [45]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# Define a FOV size that scales with the zoom factor
zoom_factor = 0.8  # Adjust this value to zoom in or out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Debugging: Inspect voxel data for a limited number of detectors
selected_detectors = [0, 1, 2]  # Change this to a smaller number of detectors as needed

for det_id in selected_detectors:
    print(f"\nInspecting Detector {det_id}")
    voxel_slice = system_matrix[det_id].reshape(fov_shape)
    print(f"Voxel slice shape: {voxel_slice.shape}")

    # Summarize counts for each plane
    xy_counts = voxel_slice.sum(axis=2)
    yz_counts = voxel_slice.sum(axis=0)
    xz_counts = voxel_slice.sum(axis=1)

    print(f"Total counts in X-Y Plane: {np.sum(xy_counts)}")
    print(f"Total counts in Y-Z Plane: {np.sum(yz_counts)}")
    print(f"Total counts in X-Z Plane: {np.sum(xz_counts)}")

    # Print statistics for the voxel slice
    print(f"XY counts min: {xy_counts.min()}, max: {xy_counts.max()}, mean: {xy_counts.mean()}")
    print(f"YZ counts min: {yz_counts.min()}, max: {yz_counts.max()}, mean: {yz_counts.mean()}")
    print(f"XZ counts min: {xz_counts.min()}, max: {xz_counts.max()}, mean: {xz_counts.mean()}")

    # Visualize aggregate counts
    plt.figure(figsize=(12, 4))

    # X-Y plane (sum across Z)
    plt.subplot(1, 3, 1)
    plt.imshow(xy_counts, cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - X-Y Plane (sum across Z)')
    plt.colorbar(label='Counts')

    # Y-Z plane (sum across X)
    plt.subplot(1, 3, 2)
    plt.imshow(yz_counts, cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - Y-Z Plane (sum across X)')
    plt.colorbar(label='Counts')

    # X-Z plane (sum across Y)
    plt.subplot(1, 3, 3)
    plt.imshow(xz_counts, cmap='viridis', aspect='auto')
    plt.title(f'Detector {det_id} - X-Z Plane (sum across Y)')
    plt.colorbar(label='Counts')

    plt.tight_layout()
    plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.

Inspecting Detector

In [49]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

# Create a 3D scatter plot of the data
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# Filter for a specific detector to visualize
detector_to_visualize = 2
mask = detector_ids == detector_to_visualize
ax.scatter(source_pos_x[mask], source_pos_y[mask], source_pos_z[mask], alpha=0.5)

ax.set_xlabel('Source Position X (mm)')
ax.set_ylabel('Source Position Y (mm)')
ax.set_zlabel('Source Position Z (mm)')
ax.set_title(f'3D Event Distribution for Detector {detector_to_visualize}')

# Save the plot as an image file
output_image_path = "/user/tridevme/MCSIM/MCSIM/macro/3D_event_distribution_detector_2.png"
plt.savefig(output_image_path)
print(f"Plot saved to {output_image_path}")

# Show the plot
plt.show()


Plot saved to /user/tridevme/MCSIM/MCSIM/macro/3D_event_distribution_detector_2.png


In [3]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# Define a FOV size and bin ranges
fov_shape = (200, 200, 100)
system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

x_bins = np.linspace(np.min(source_pos_x), np.max(source_pos_x), fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y), np.max(source_pos_y), fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z), np.max(source_pos_z), fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Choose slice based on user input
    if axis == 'Z':
        slice_data = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Z_slice_zoomed.png"
    
    elif axis == 'X':
        # Option 1: Take a wider average along the X-axis (3/4 width instead of the previous 1/2)
        slice_data = np.mean(voxel_slice[fov_shape[0] // 8:7 * fov_shape[0] // 8, :, :], axis=0)

        # Option 2: Uncomment the line below to use a single central slice
        # slice_data = voxel_slice[fov_shape[0] // 2, :, :]

        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_zoomed.png"
    
    elif axis == 'Y':
        slice_data = np.mean(voxel_slice[:, fov_shape[1] // 4:3 * fov_shape[1] // 4, :], axis=1)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Y_slice_zoomed.png"
    
    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    # Use a logarithmic color scale for the plot
    im = ax.imshow(slice_data, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(slice_data) or 1))
    ax.set_title(f'Detector {det_id}')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Enter the axis for s

In [13]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.ndimage import gaussian_filter

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Define FOV and bin sizes for zooming and smoothing
zoom_factor = 0.3  # Use values < 1 to zoom in, values > 1 to zoom out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Choose slice based on user input
    if axis == 'Z':
        slice_data = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Z_slice_contour.png"
    
    elif axis == 'X':
        slice_data = np.mean(voxel_slice[fov_shape[0] // 4:3 * fov_shape[0] // 4, :, :], axis=0)
        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_contour.png"
    
    elif axis == 'Y':
        slice_data = np.mean(voxel_slice[:, fov_shape[1] // 4:3 * fov_shape[1] // 4, :], axis=1)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macfro/system_matrix_multiple_detectors_Y_slice_contour.png"
    
    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    # Smooth the image data to enhance the structure
    slice_data_smoothed = gaussian_filter(slice_data, sigma=1)

    # Plot the smoothed slice with transparency and contour overlay
    im = ax.imshow(slice_data_smoothed, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(slice_data_smoothed) or 1), alpha=0.7)
    ax.contour(slice_data_smoothed, levels=5, colors='white', linewidths=0.5)

    ax.set_title(f'Detector {det_id}')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated.
Enter the axis for slicing (X, Y, or Z): X
Plot saved to /user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_c

In [8]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.ndimage import sobel

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Check event distribution across detectors
unique_detectors, counts = np.unique(detector_ids, return_counts=True)
for det, count in zip(unique_detectors, counts):
    print(f"Detector {det} has {count} events.")

# Define a FOV size that scales with the zoom factor
zoom_factor = 1  # Use values < 1 to zoom in, values > 1 to zoom out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Choose slice based on user input
    if axis == 'Z':
        slice_data = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Z_slice_zoomed_edge.png"
    
    elif axis == 'X':
        slice_data = np.mean(voxel_slice[fov_shape[0] // 4:3 * fov_shape[0] // 4, :, :], axis=0)
        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_zoomed_edge.png"
    
    elif axis == 'Y':
        slice_data = np.mean(voxel_slice[:, fov_shape[1] // 4:3 * fov_shape[1] // 4, :], axis=1)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Y_slice_zoomed_edge.png"
    
    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    # Apply Sobel edge detection to emphasize contours
    edge_data = sobel(slice_data)
    im = ax.imshow(edge_data, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(edge_data) or 1))
    ax.set_title(f'Detector {det_id} - Edge Detection')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
Detector 0 has 9611 events.
Detector 1 has 2580 events.
Detector 2 has 9704 events.
Detector 3 has 2600 events.
System matrix generated.
Enter the axis for s

In [15]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.ndimage import sobel

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Define a FOV size that scales with the zoom factor
zoom_factor = 0.2  # Use values < 1 to zoom in, values > 1 to zoom out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector slice
fig, axs = plt.subplots(2, 4, figsize=(16, 12))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx // 4, idx % 4]
    voxel_slice = system_matrix[det_id].reshape(fov_shape)

    # Choose slice based on user input
    if axis == 'Z':
        slice_data = np.mean(voxel_slice[:, :, fov_shape[2] // 4:3 * fov_shape[2] // 4], axis=2)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view y-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Z_slice_zoomed_overlay.png"
    
    elif axis == 'X':
        slice_data = np.mean(voxel_slice[fov_shape[0] // 4:3 * fov_shape[0] // 4, :, :], axis=0)
        ax.set_xlabel('Field of view y-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_zoomed_overlay.png"
    
    elif axis == 'Y':
        slice_data = np.mean(voxel_slice[:, fov_shape[1] // 4:3 * fov_shape[1] // 4, :], axis=1)
        ax.set_xlabel('Field of view x-axis (mm)')
        ax.set_ylabel('Field of view z-axis (mm)')
        output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_Y_slice_zoomed_overlay.png"
    
    else:
        print("Invalid axis selected. Please choose X, Y, or Z.")
        break

    # Apply Sobel edge detection to emphasize contours and amplify the edges
    edge_data = sobel(slice_data)
    edge_data = np.clip(edge_data * 5, 0, np.max(edge_data))  # Increase contrast

    # Overlay edges on the original data by combining the original and edge-detected images
    combined_data = slice_data + edge_data
    im = ax.imshow(combined_data, cmap='viridis', origin='lower', norm=LogNorm(vmin=1e-4, vmax=np.max(combined_data) or 1))
    ax.set_title(f'Detector {det_id} - Edge Overlay')

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
plt.savefig(output_path)
print(f"Plot saved to {output_path}")
plt.show()


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated.
Enter the axis for slicing (X, Y, or Z): X
Plot saved to /user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_X_slice_z

In [36]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.ndimage import gaussian_filter

# Load the ROOT file
file_path = "/user/tridevme/MCSIM/MCSIM/macro/SPEBT.Singles_merged.root"
root_file = ROOT.TFile(file_path)
tree = root_file.Get("tree")

# Initialize lists to store data
detector_ids = []
source_pos_x = []
source_pos_y = []
source_pos_z = []

# Process entries in batches to avoid memory issues
batch_size = 1000
n_entries = tree.GetEntries()

for entry in range(0, n_entries, batch_size):
    max_entry = min(entry + batch_size, n_entries)
    for i in range(entry, max_entry):
        tree.GetEntry(i)
        detector_ids.append(tree.crystalID)
        source_pos_x.append(tree.sourcePosX)
        source_pos_y.append(tree.sourcePosY)
        source_pos_z.append(tree.sourcePosZ)
    print(f"Processed entries {entry} to {max_entry}")

print("Data processed and converted to arrays.")

# Convert lists to arrays
detector_ids = np.array(detector_ids)
source_pos_x = np.array(source_pos_x)
source_pos_y = np.array(source_pos_y)
source_pos_z = np.array(source_pos_z)

# Define FOV and bin sizes for zooming and smoothing
zoom_factor = 4  # Use values < 1 to zoom in, values > 1 to zoom out
fov_shape = (int(200 * zoom_factor), int(200 * zoom_factor), int(100 * zoom_factor))

system_matrix = np.zeros((len(np.unique(detector_ids)), np.prod(fov_shape)))

# Adjust bins based on zoomed FOV
x_bins = np.linspace(np.min(source_pos_x) - 50 * zoom_factor, np.max(source_pos_x) + 50 * zoom_factor, fov_shape[0])
y_bins = np.linspace(np.min(source_pos_y) - 50 * zoom_factor, np.max(source_pos_y) + 50 * zoom_factor, fov_shape[1])
z_bins = np.linspace(np.min(source_pos_z) - 50 * zoom_factor, np.max(source_pos_z) + 50 * zoom_factor, fov_shape[2])

for i, det_id in enumerate(detector_ids):
    x_idx = np.digitize(source_pos_x[i], x_bins) - 1
    y_idx = np.digitize(source_pos_y[i], y_bins) - 1
    z_idx = np.digitize(source_pos_z[i], z_bins) - 1
    if 0 <= x_idx < fov_shape[0] and 0 <= y_idx < fov_shape[1] and 0 <= z_idx < fov_shape[2]:
        voxel_idx = np.ravel_multi_index((x_idx, y_idx, z_idx), fov_shape)
        system_matrix[det_id, voxel_idx] += 1

print("System matrix generated.")

# Get the axis for slicing from the user
axis = input("Enter the axis for slicing (X, Y, or Z): ").strip().upper()

# Select detectors for visualization
selected_detectors = [0, 1, 2, 3]
n_detectors = len(selected_detectors)

# Create subplots for each selected detector scatter plot
fig, axs = plt.subplots(1, 4, figsize=(20, 6))

for idx, det_id in enumerate(selected_detectors):
    ax = axs[idx]
    
    # Get only the data for the selected detector
    mask = detector_ids == det_id
    x_data = source_pos_x[mask]
    y_data = source_pos_y[mask]
    z_data = source_pos_z[mask]
    
    # Plot scatter points
    sc = ax.scatter(y_data, z_data, c=x_data, s=1, cmap='viridis', norm=LogNorm(vmin=1e-4, vmax=1e-1), alpha=0.6)
    
    ax.set_title(f'Detector {det_id}')
    ax.set_xlabel("Field of view y-axis (mm)")
    ax.set_ylabel("Field of view z-axis (mm)")

# Adjust layout and add a colorbar
fig.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(sc, cax=cbar_ax)
cbar.set_label('Logarithmic scale')

# Save the plot
output_path = "/user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_scatter.png"
plt.savefig(output_path)
print(f"Plot saved to {output_path}")


Processed entries 0 to 1000
Processed entries 1000 to 2000
Processed entries 2000 to 3000
Processed entries 3000 to 4000
Processed entries 4000 to 5000
Processed entries 5000 to 6000
Processed entries 6000 to 7000
Processed entries 7000 to 8000
Processed entries 8000 to 9000
Processed entries 9000 to 10000
Processed entries 10000 to 11000
Processed entries 11000 to 12000
Processed entries 12000 to 13000
Processed entries 13000 to 14000
Processed entries 14000 to 15000
Processed entries 15000 to 16000
Processed entries 16000 to 17000
Processed entries 17000 to 18000
Processed entries 18000 to 19000
Processed entries 19000 to 20000
Processed entries 20000 to 21000
Processed entries 21000 to 22000
Processed entries 22000 to 23000
Processed entries 23000 to 24000
Processed entries 24000 to 24495
Data processed and converted to arrays.
System matrix generated.
Enter the axis for slicing (X, Y, or Z): Y
Plot saved to /user/tridevme/MCSIM/MCSIM/macro/system_matrix_multiple_detectors_scatter.p